# Optimization of Ellipsometric measurements



**Optimization Algorithm**: 

Applying SCE UA (Shuffled complex evolution optimization algorithm) by [Duan et al](https://www.sciencedirect.com/science/article/pii/0022169494900574) with the modification of python implementation of 
[Stijn Van Hoey ](https://github.com/stijnvanhoey/Optimization_SCE)





In [1]:
import numpy as np
from random import randrange
import SCE

## 1. Helpful functions

In [2]:
def angle2ST(x,s,t):  
    
    # x: array of (azimuth, ellipsity), len(x) depends on s and t
    # s,t: number of PSG state and PSA state, respectively
    
    # number of parameters=2*(s + t)    
    # case 1: 1 PSG + 4 PSA: 10 parameters
    # case 2: 2 PSG + 2PSA: 8 parameters    
    
    angleS=np.array([[x[i],x[s+t+i]] for i in range(s+t) if i<s])
    angleT=np.array([[x[i],x[s+t+i]] for i in range(s+t) if i>=s])
    
    vectorS=Fvector(angleS)
    vectorT=Fvector(angleT)
    
    return vectorS, vectorT   


def Fvector(angle):
    
    def vectori(alpi,epsi): # vector element
        alphai=np.radians(alpi) # covert to radians
        epsiloni=np.radians(epsi)
        return np.array([[1,np.cos(2*alphai)*np.cos(2*epsiloni),np.sin(2*alphai)*np.cos(2*epsiloni),np.sin(2*epsiloni)]]).T
    
    vector=vectori(angle.item(0,0),angle.item(0,1)) # initializing vector
    if len(angle) >1:
        for i in range(1,len(angle)):
            s1=angle.item(i,0)
            s2=angle.item(i,1)
            vector=np.append(vector,vectori(s1,s2),axis=1)   
           
    return vector

# Define function to optimize

def roundmatrix(M): # round all element of matrix
    for i in range (0,M.shape[0]):
        for j in range(0, M.shape[1]):
            M.itemset((i,j), np.round(M.item(i,j),5))    

## 2. Start to optimize

In [3]:
def Ellipsometry(OptFunction,number,s,t):
    
    #OptFunction: function for the optimization
    # number: number of run (with different x0)
    
    # s,t: number of PSG state and PSA state, respectively, defining the dimension of starting point x0   
    
    # parameters for SCEUA optimazation
    maxn=100000
    kstop=1000
    pcento=1.e-5
    peps=0.001
    iseed= 0
    iniflg=0
    ngs=5

    # bound for each variable
    pi=180
    bl=np.concatenate(([-pi/2]*(s+t),[-pi/4]*(s+t)),axis=0)
    bu=np.concatenate(([pi/2]*(s+t),[pi/4]*(s+t)),axis=0)


    # run optimization with number values of  x0

    Listbestx=[];Listbestf=[]
    List_vectorS=[];List_vectorT=[]

    for i in range(number):
        #starting with a random x0
        x0=np.concatenate(([randrange(-90,90,1) for i in range(s+t)],[randrange(-45,45,1) for i in range(s+t)]),axis=0)

        bestx,bestf,BESTX,BESTF,ICALL = SCE.sceua(x0,bl,bu,maxn,kstop,pcento,peps,ngs,iseed,iniflg,OptFunction)

        Listbestx.append(bestx)
        Listbestf.append(bestf)
    
        S,T=angle2ST(bestx,s,t)
    
        roundmatrix(S)
        roundmatrix(T)
        List_vectorS.append(S)
        List_vectorT.append(T)
    return List_vectorS,List_vectorT,Listbestx,Listbestf
    

## 2.1 Case 1:  one  PSG state + four  PSA states

In [4]:
# define function to optimize
# case 1: s=1, t=4
def OptFunction1(x):      
   
    vectorS,vectorT=angle2ST(x,1,4)    
    
    W=np.array([
    [1+vectorS.item(1,0)*vectorT.item(1,0),vectorT.item(1,0)+vectorS.item(1,0),vectorS.item(2,0)*vectorT.item(2,0)+vectorS.item(3,0)*vectorT.item(3,0),vectorS.item(3,0)*vectorT.item(2,0)-vectorS.item(2,0)*vectorT.item(3,0) ],
    [1+vectorS.item(1,0)*vectorT.item(1,1),vectorT.item(1,1)+vectorS.item(1,0),vectorS.item(2,0)*vectorT.item(2,1)+vectorS.item(3,0)*vectorT.item(3,1),vectorS.item(3,0)*vectorT.item(2,1)-vectorS.item(2,0)*vectorT.item(3,1) ],
    [1+vectorS.item(1,0)*vectorT.item(1,2),vectorT.item(1,2)+vectorS.item(1,0),vectorS.item(2,0)*vectorT.item(2,2)+vectorS.item(3,0)*vectorT.item(3,2),vectorS.item(3,0)*vectorT.item(2,2)-vectorS.item(2,0)*vectorT.item(3,2) ],
    [1+vectorS.item(1,0)*vectorT.item(1,3),vectorT.item(1,3)+vectorS.item(1,0),vectorS.item(2,0)*vectorT.item(2,3)+vectorS.item(3,0)*vectorT.item(3,3),vectorS.item(3,0)*vectorT.item(2,3)-vectorS.item(2,0)*vectorT.item(3,3) ]
])
           
    W_inv=np.linalg.inv(W)
    Gauss=np.dot(W_inv,W_inv.T)
    
    f = np.trace(Gauss)
    
    return f            

In [5]:
# Case 1
#number =1, s=1, t=4
List_vectorS_1,List_vectorT_1,List_bestx_1,List_bestf_1=Ellipsometry(OptFunction1,1,1,4)

The Initial Loop: 0
 BESTF:  3.775659 
 BESTX:  
[-37.25634879  62.80983996  21.21780455 -87.6173656  -27.49796677
 -31.66732251  43.36464508  -1.94667237  -0.23477711  12.55252648]
 WORSTF:  17492933.506225 
 WORSTX: 
[-57.39282689  51.93819222 -79.76734624  35.45950351  50.16517127
  24.96668057 -21.65196921 -11.35681759   7.88396717 -20.44602878]
     
Evolution Loop: 1  - Trial - 278
 BESTF:  3.775659 
 BESTX:  
[-37.25634879  62.80983996  21.21780455 -87.6173656  -27.49796677
 -31.66732251  43.36464508  -1.94667237  -0.23477711  12.55252648]
 WORSTF:  17492933.506225 
 WORSTX: 
[-57.39282689  51.93819222 -79.76734624  35.45950351  50.16517127
  24.96668057 -21.65196921 -11.35681759   7.88396717 -20.44602878]
     
Evolution Loop: 2  - Trial - 424
 BESTF:  3.775659 
 BESTX:  
[-37.25634879  62.80983996  21.21780455 -87.6173656  -27.49796677
 -31.66732251  43.36464508  -1.94667237  -0.23477711  12.55252648]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8

Evolution Loop: 27  - Trial - 3741
 BESTF:  3.746596 
 BESTX:  
[-50.36527044  16.9192245   26.79765964  79.1347406  -20.78362663
   1.34250349  27.58380972 -28.1361515   -9.26149872  -2.67930911]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 28  - Trial - 3864
 BESTF:  3.746596 
 BESTX:  
[-50.36527044  16.9192245   26.79765964  79.1347406  -20.78362663
   1.34250349  27.58380972 -28.1361515   -9.26149872  -2.67930911]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 29  - Trial - 3984
 BESTF:  3.746596 
 BESTX:  
[-50.36527044  16.9192245   26.79765964  79.1347406  -20.78362663
   1.34250349  27.58380972 -28.1361515   -9.26149872  -2.67930911]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.4

Evolution Loop: 55  - Trial - 7173
 BESTF:  3.746596 
 BESTX:  
[-50.36527044  16.9192245   26.79765964  79.1347406  -20.78362663
   1.34250349  27.58380972 -28.1361515   -9.26149872  -2.67930911]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 56  - Trial - 7294
 BESTF:  3.746596 
 BESTX:  
[-50.36527044  16.9192245   26.79765964  79.1347406  -20.78362663
   1.34250349  27.58380972 -28.1361515   -9.26149872  -2.67930911]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 57  - Trial - 7423
 BESTF:  3.746596 
 BESTX:  
[-50.36527044  16.9192245   26.79765964  79.1347406  -20.78362663
   1.34250349  27.58380972 -28.1361515   -9.26149872  -2.67930911]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.4

Evolution Loop: 79  - Trial - 10034
 BESTF:  3.746596 
 BESTX:  
[-50.36527044  16.9192245   26.79765964  79.1347406  -20.78362663
   1.34250349  27.58380972 -28.1361515   -9.26149872  -2.67930911]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 80  - Trial - 10149
 BESTF:  3.746596 
 BESTX:  
[-50.36527044  16.9192245   26.79765964  79.1347406  -20.78362663
   1.34250349  27.58380972 -28.1361515   -9.26149872  -2.67930911]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 81  - Trial - 10270
 BESTF:  3.746596 
 BESTX:  
[-50.36527044  16.9192245   26.79765964  79.1347406  -20.78362663
   1.34250349  27.58380972 -28.1361515   -9.26149872  -2.67930911]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  1

Evolution Loop: 102  - Trial - 12729
 BESTF:  3.746596 
 BESTX:  
[-50.36527044  16.9192245   26.79765964  79.1347406  -20.78362663
   1.34250349  27.58380972 -28.1361515   -9.26149872  -2.67930911]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 103  - Trial - 12844
 BESTF:  3.746596 
 BESTX:  
[-50.36527044  16.9192245   26.79765964  79.1347406  -20.78362663
   1.34250349  27.58380972 -28.1361515   -9.26149872  -2.67930911]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 104  - Trial - 12962
 BESTF:  3.746596 
 BESTX:  
[-50.36527044  16.9192245   26.79765964  79.1347406  -20.78362663
   1.34250349  27.58380972 -28.1361515   -9.26149872  -2.67930911]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595

...

Evolution Loop: 793  - Trial - 88158
 BESTF:  2.500000 
 BESTX:  
[-45.00000006   7.02114193 -27.56835257  63.38131725 -60.11642632
   4.71808029  13.51755531 -35.92747869   0.66828662  14.09540143]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 794  - Trial - 88265
 BESTF:  2.500000 
 BESTX:  
[-45.00000006   7.02114193 -27.56835257  63.38131725 -60.11642632
   4.71808029  13.51755531 -35.92747869   0.66828662  14.09540143]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 795  - Trial - 88371
 BESTF:  2.500000 
 BESTX:  
[-45.00000006   7.02114193 -27.56835257  63.38131725 -60.11642632
   4.71808029  13.51755531 -35.92747869   0.66828662  14.09540143]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595

Evolution Loop: 816  - Trial - 90613
 BESTF:  2.500000 
 BESTX:  
[-45.00000006   7.02114193 -27.56835257  63.38131725 -60.11642632
   4.71808029  13.51755531 -35.92747869   0.66828662  14.09540143]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 817  - Trial - 90719
 BESTF:  2.500000 
 BESTX:  
[-45.00000006   7.02114193 -27.56835257  63.38131725 -60.11642632
   4.71808029  13.51755531 -35.92747869   0.66828662  14.09540143]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 818  - Trial - 90824
 BESTF:  2.500000 
 BESTX:  
[-45.00000006   7.02114193 -27.56835257  63.38131725 -60.11642632
   4.71808029  13.51755531 -35.92747869   0.66828662  14.09540143]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595

Evolution Loop: 839  - Trial - 93057
 BESTF:  2.500000 
 BESTX:  
[-45.00000006   7.02114193 -27.56835257  63.38131725 -60.11642632
   4.71808029  13.51755531 -35.92747869   0.66828662  14.09540143]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 840  - Trial - 93163
 BESTF:  2.500000 
 BESTX:  
[-45.00000006   7.02114193 -27.56835257  63.38131725 -60.11642632
   4.71808029  13.51755531 -35.92747869   0.66828662  14.09540143]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 841  - Trial - 93270
 BESTF:  2.500000 
 BESTX:  
[-45.00000006   7.02114193 -27.56835257  63.38131725 -60.11642632
   4.71808029  13.51755531 -35.92747869   0.66828662  14.09540143]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595

Evolution Loop: 863  - Trial - 95618
 BESTF:  2.500000 
 BESTX:  
[-45.00000006   7.02114193 -27.56835257  63.38131725 -60.11642632
   4.71808029  13.51755531 -35.92747869   0.66828662  14.09540143]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 864  - Trial - 95723
 BESTF:  2.500000 
 BESTX:  
[-45.00000006   7.02114193 -27.56835257  63.38131725 -60.11642632
   4.71808029  13.51755531 -35.92747869   0.66828662  14.09540143]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 865  - Trial - 95828
 BESTF:  2.500000 
 BESTX:  
[-45.00000006   7.02114193 -27.56835257  63.38131725 -60.11642632
   4.71808029  13.51755531 -35.92747869   0.66828662  14.09540143]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595

Evolution Loop: 886  - Trial - 98062
 BESTF:  2.500000 
 BESTX:  
[-45.00000006   7.02114193 -27.56835257  63.38131725 -60.11642632
   4.71808029  13.51755531 -35.92747869   0.66828662  14.09540143]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 887  - Trial - 98167
 BESTF:  2.500000 
 BESTX:  
[-45.00000006   7.02114193 -27.56835257  63.38131725 -60.11642632
   4.71808029  13.51755531 -35.92747869   0.66828662  14.09540143]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595  18.49740769   8.07897294 -13.74213612
  13.13047018  -5.61715099  35.25957007  41.72964845 -10.49026331]
     
Evolution Loop: 888  - Trial - 98275
 BESTF:  2.500000 
 BESTX:  
[-45.00000006   7.02114193 -27.56835257  63.38131725 -60.11642632
   4.71808029  13.51755531 -35.92747869   0.66828662  14.09540143]
 WORSTF:  2656.609082 
 WORSTX: 
[  8.78643071  38.73408595

## 2.2 Case 2: two  PSG states + two  PSA states

In [6]:
# define function to optimize
# case 2: s=2, t=2
def OptFunction2(x):      
   
    vectorS,vectorT=angle2ST(x,2,2)    
    
    W=np.array([
    [1+vectorS.item(1,0)*vectorT.item(1,0),vectorT.item(1,0)+vectorS.item(1,0),vectorS.item(2,0)*vectorT.item(2,0)+vectorS.item(3,0)*vectorT.item(3,0),vectorS.item(3,0)*vectorT.item(2,0)-vectorS.item(2,0)*vectorT.item(3,0) ],
    [1+vectorS.item(1,0)*vectorT.item(1,1),vectorT.item(1,1)+vectorS.item(1,0),vectorS.item(2,0)*vectorT.item(2,1)+vectorS.item(3,0)*vectorT.item(3,1),vectorS.item(3,0)*vectorT.item(2,1)-vectorS.item(2,0)*vectorT.item(3,1) ],
    [1+vectorS.item(1,1)*vectorT.item(1,0),vectorT.item(1,0)+vectorS.item(1,1),vectorS.item(2,1)*vectorT.item(2,0)+vectorS.item(3,1)*vectorT.item(3,0),vectorS.item(3,1)*vectorT.item(2,0)-vectorS.item(2,1)*vectorT.item(3,0) ],
    [1+vectorS.item(1,1)*vectorT.item(1,1),vectorT.item(1,1)+vectorS.item(1,1),vectorS.item(2,1)*vectorT.item(2,1)+vectorS.item(3,1)*vectorT.item(3,1),vectorS.item(3,1)*vectorT.item(2,1)-vectorS.item(2,1)*vectorT.item(3,1) ]
])
           
    W_inv=np.linalg.inv(W)
    Gauss=np.dot(W_inv,W_inv.T)
    
    f = np.trace(Gauss)
    
    return f            

In [7]:
# Case 2
#number =1, s=2, t=2
List_vectorS_2,List_vectorT_2,List_bestx_2,List_bestf_2=Ellipsometry(OptFunction2,1,2,2)

The Initial Loop: 0
 BESTF:  3.459091 
 BESTX:  
[-77.24343993 -37.29707434 -62.57615298 -14.85245254 -33.18396044
   9.37060236 -10.54727468  35.58472959]
 WORSTF:  336121.459745 
 WORSTX: 
[53.35046541 82.64998855 -7.53501109 16.37714976 32.19503798 -3.8498892
 40.66870291  6.81760458]
     
Evolution Loop: 1  - Trial - 214
 BESTF:  3.459091 
 BESTX:  
[-77.24343993 -37.29707434 -62.57615298 -14.85245254 -33.18396044
   9.37060236 -10.54727468  35.58472959]
 WORSTF:  12885.707441 
 WORSTX: 
[ 84.41490684  72.8907022  -36.61987229  88.56202381 -22.5521963
 -35.46844606  40.58573499 -23.99217701]
     
Evolution Loop: 2  - Trial - 359
 BESTF:  3.090313 
 BESTX:  
[-15.35003647 -61.50811255 -43.32505955  -9.03061604  16.55252923
 -12.52633224  24.44146471 -44.13202027]
 WORSTF:  7213.370428 
 WORSTX: 
[-60.75127176  20.80072157 -67.71240309  62.64148128  27.65870629
   6.21906648  -8.35350325 -38.77497041]
     
Evolution Loop: 3  - Trial - 491
 BESTF:  3.090313 
 BESTX:  
[-15.35003647

Evolution Loop: 27  - Trial - 2803
 BESTF:  2.601221 
 BESTX:  
[-62.88505387 -21.70399368  44.90760535 -48.45905967  12.06613373
 -24.41227149  24.40026413 -23.3105282 ]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 28  - Trial - 2892
 BESTF:  2.601221 
 BESTX:  
[-62.88505387 -21.70399368  44.90760535 -48.45905967  12.06613373
 -24.41227149  24.40026413 -23.3105282 ]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 29  - Trial - 2984
 BESTF:  2.601221 
 BESTX:  
[-62.88505387 -21.70399368  44.90760535 -48.45905967  12.06613373
 -24.41227149  24.40026413 -23.3105282 ]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 30  - Trial - 3080
 BESTF:  2.601221 
 BESTX:  
[-62.88505387 -

Evolution Loop: 57  - Trial - 5755
 BESTF:  2.500031 
 BESTX:  
[-64.74241079 -17.94856303  45.15491401 -44.9896911   12.31870001
 -22.30259541  23.63225645 -23.60535302]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 58  - Trial - 5868
 BESTF:  2.500024 
 BESTX:  
[-64.678373   -17.82481653  44.95968848 -44.99385304  12.32565853
 -22.30265483  23.54311089 -23.65123869]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 59  - Trial - 5973
 BESTF:  2.500014 
 BESTX:  
[-64.73038576 -17.9808827   45.0744476  -44.96435093  12.33500895
 -22.28620286  23.61857158 -23.63110472]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 60  - Trial - 6081
 BESTF:  2.500006 
 BESTX:  
[-64.66916847 -

Evolution Loop: 87  - Trial - 8952
 BESTF:  2.500000 
 BESTX:  
[-64.72003445 -17.92531573  45.00003886 -44.99996042  12.32820676
 -22.28888849  23.65698942 -23.65693135]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 88  - Trial - 9055
 BESTF:  2.500000 
 BESTX:  
[-64.71997784 -17.92527505  44.99996967 -45.00001357  12.32808258
 -22.28895521  23.65665738 -23.65666137]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 89  - Trial - 9162
 BESTF:  2.500000 
 BESTX:  
[-64.71998501 -17.92527706  44.99997644 -45.00003927  12.32814363
 -22.28894513  23.65635382 -23.65642863]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 90  - Trial - 9265
 BESTF:  2.500000 
 BESTX:  
[-64.71997843 -

Evolution Loop: 116  - Trial - 11992
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 117  - Trial - 12090
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 118  - Trial - 12184
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 119  - Trial - 12276
 BESTF:  2.500000 
 BESTX:  
[-64.72

...

Evolution Loop: 886  - Trial - 86758
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 887  - Trial - 86858
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 888  - Trial - 86951
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 889  - Trial - 87052
 BESTF:  2.500000 
 BESTX:  
[-64.72

Evolution Loop: 919  - Trial - 89978
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 920  - Trial - 90074
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 921  - Trial - 90169
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 922  - Trial - 90266
 BESTF:  2.500000 
 BESTX:  
[-64.72

Evolution Loop: 952  - Trial - 93152
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 953  - Trial - 93250
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 954  - Trial - 93348
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 955  - Trial - 93447
 BESTF:  2.500000 
 BESTX:  
[-64.72

Evolution Loop: 985  - Trial - 96392
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 986  - Trial - 96490
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 987  - Trial - 96588
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 988  - Trial - 96688
 BESTF:  2.500000 
 BESTX:  
[-64.72

Evolution Loop: 1017  - Trial - 99512
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 1018  - Trial - 99609
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 1019  - Trial - 99702
 BESTF:  2.500000 
 BESTX:  
[-64.7200492  -17.92542896  45.00000005 -45.00000019  12.32826569
 -22.28884138  23.65698525 -23.65698586]
 WORSTF:  94.863580 
 WORSTX: 
[ 8.78643071 38.73408595 18.49740769  8.07897294 -6.87106806 13.13047018
 -5.61715099 35.25957007]
     
Evolution Loop: 1020  - Trial - 99797
 BESTF:  2.500000 
 BESTX:  
[-6